In [1]:
import importlib
import tidy_data
importlib.reload(tidy_data)
from tidy_data import profile_stats, process_skills, get_long_jobs, get_long_education

import pandas as pd
pd.set_option('display.max_rows', 500)
import datetime
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import collections
%matplotlib inline
from scipy import stats

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
with open('data/raw/scraped_profiles.json') as json_data:
    profiles = json_data.readlines()

profiles_df = pd.read_csv('profiles_df.csv'); print(len(profiles_df))

21060


In [3]:
jobs = get_long_jobs(profiles); len(jobs)

165831

### Seniority
    - Entry level:
        - Junior
        - assistant
        - intern
        - trainee
        - associate

    - Mid level:
        - Account manager
        - Team lead
        - Accounting officer
        - Project superintendent
        - Regional manager
        - IT supervisor

    - Senior level:
        - Executive director
        - Vice president
        - Chief financial officer
        - Head of advertising
        - HR director
        - Senior architect

Keywords
- Executive
- Director
- Manager
- Senior level
- Associate
- Entry level
- Junior
- Internship

In [4]:
junior_roles = "|".join(['junior','assistant','\\bintern\\b','\\binternship\\b','trainee','associate','jr'])
mid_roles = "|".join(['account manager','project manager','lead','manager','supervisor','project director'])
senior_roles_chief = "|".join(['\\bc[a-z]o\\b','chief'])
senior_roles_other = "|".join(['executive','director','head',
                         'director','senior','sr','principal'])
board_roles = "|".join(['president','vp','board','chairman'])
founder = "|".join(['founder','partner','owner'])

In [5]:
# Clean occupation titles (i.e. lowercase, remove stop words)
def process_occupation(occupation):
    text = occupation.lower()
    
    remove_punct = str.maketrans("", "", string.punctuation)
    text = text.translate(remove_punct)
    
    remove_digits = str.maketrans("", "", string.digits)
    text = text.translate(remove_digits)
    
    text = text.replace(' ml ', 'machine learning')
    text = text.replace(' ai ', 'artificial intelligence')
    
    tokens = word_tokenize(text)
    
    stop_words = stopwords.words('english')
    stop_words.remove('it')
    tokens_stop = [y for y in tokens if y not in stop_words]
    text = " ".join(tokens_stop)
    text2, text3 = '', ''

    if len(re.findall(founder, text)) > 0:
        text2 = re.findall(founder, text)[0]
        text3 = "Founder"
        
        
    elif len(re.findall(board_roles, text)) > 0:
        text2 = re.findall(board_roles, text)[0]
        text3 = "Board"

    
    elif len(re.findall(senior_roles_chief, text)) > 0:
        text2 = "CXO"
        text3 = "Senior"
    
    elif len(re.findall(senior_roles_other, text)) > 0:
        text2 = re.findall(senior_roles_other, text)[0]
        if text2 == 'sr':
            text2 = 'senior'
        text3 = "Senior"
    
    elif len(re.findall(mid_roles, text)) > 0:
        text2 = re.findall(mid_roles, text)[0]
        text3 = "Mid"

    
    elif len(re.findall(junior_roles, text)) > 0:
        text2 = re.findall(junior_roles, text)[0]
        if text2 == 'jr':
            text2 = 'junior'
        text3 = "Junior"

    return text, text2, text3

In [17]:
role_clean, seniority, seniority2 = [],[],[]
for role in jobs.role.values:
    out = process_occupation(role)
    role_clean.append(out[0])
    seniority.append(out[1])
    seniority2.append(out[2])

In [18]:
jobs['role_clean'], jobs['seniority'], jobs['seniority2'] = role_clean, seniority, seniority2

In [207]:
len(jobs[jobs.seniority2.isin(['Senior','Mid','Junior','Founder','Board'])])

128805

In [124]:
len(jobs[jobs.seniority == 'Junior'])

4359

In [197]:
for idx, r in enumerate(set(jobs[jobs.seniority2 == 'Mid'].role_clean.values)):
    print(idx, r)

0 infrastructure manager global project manager
1 development manager wwwfi
2 innovation brand manager veet europe north america ena
3 technical manager valueadded services engineer
4 manager business intelligence data warehouse
5 customer care manager hr manager
6 country manager ch italy
7 manager business information systems competence centre
8 innovation lead dach
9 supply chain reporting capability manager
10 lead data scientist research
11 frenchamerican foundation young leaders program member
12 manager stability sciences
13 marketing manager innovation
14 iot project manager
15 digital offering manager atos origin france
16 ferrari maserati mechanical chassis components systems project manager
17 lead product manager digital transformation contract
18 project manager data management programme
19 skill group manager
20 fanatical aws commercial manager emea
21 product lead ecommerce order management risk analytics
22 manager strategy portfolio
23 national channel manager
24 custo

952 change manager secondment
953 directeur de projet programme manager it
954 technical program manager
955 programme manager finance transformation global finance programme
956 net project manager
957 country manager sweden
958 european technical presales manager
959 department manager electrical embedded systems
960 portolio manager
961 marketing services manager
962 manager peoplesoft applications
963 program manager architect skype
964 manager sales policy management system
965 product managerindustry business consultantapplications consultant
966 product manager mobile portals applications mobile advertising
967 erp deployment manager
968 technical lead big data analytics
969 department manager data science governance rd
970 section leader rd quality control manager
971 process development manager range supply
972 lead it product management shop
973 fromageries bel service manager sap bw
974 manager it infrastructure architecture emea
975 manager supplier production management
97

1702 rd manager assa ab assa industri ab
1703 team manager applications web servers
1704 general managerlogistics operations manager
1705 business programme manager
1706 practice manager norway
1707 marketing manager crdm central eastern europe
1708 regional manager southeast asia
1709 pharmacist manager
1710 it retail manager
1711 revenue manager benelux
1712 yk programme manager
1713 deal engagement manager digital missioncritical communication systems tetra
1714 technology program process leader global product development
1715 it services manager network management backup storage service contract european commission
1716 project manager vehicle dynamic functionssw
1717 manager production validation
1718 aftersales manager
1719 strategy manager strategy office
1720 hr manager sweden
1721 greater london south finance manager
1722 general manager information security
1723 security system manager
1724 smartcard solutions project manager
1725 manager sales joint venture strategy
1726 net

2702 customer marketing manager dafalgan belux
2703 business manager automation software
2704 data manager business intelligence specialist
2705 marketing manager fixed line products
2706 test tools manager athens olympic games
2707 global marketing intelligence manager
2708 mcommerce projet manager
2709 emea business process manager
2710 verification manager
2711 project leader transition manager
2712 enterprise marketing audience marketing manager
2713 business intelligence corporate planning financial consolidations lead
2714 program customer integrations manager innovation it assets
2715 it manager field services desktop engineering
2716 operations gate manager
2717 freelance project manager devoteam nokia vodafone italy
2718 team leader vehicle access control protection systems
2719 applications manager
2720 rf project manager modem equipment earth stations
2721 global marketing manager internet ai
2722 regional manager customer intelligence
2723 data warehouse manager
2724 acting

3702 manager document content laboratory
3703 insurance segment global leader financial services
3704 assistant manager next generation services development
3705 team lead iaa conference new mobility world convention
3706 runbookmanager
3707 consulting engineer manager
3708 culinary development manager
3709 lead solution architect information excellence
3710 emerging technology manager
3711 project manager big data analytics sas bi
3712 general manager jupiter india composites private ltd
3713 manager global market intelligence global marketing systems business applications
3714 ee project manager
3715 managerartificial intelligence machine learning
3716 european merger acquisition bi program lead
3717 channel marketing manager commercial dach
3718 support manager germany northwest region
3719 programme assurance manager
3720 general manager electricalelectronic engineering software
3721 manager data engineering application hosting
3722 engineering manager crm
3723 channel manager
3724

...

10450 international development manager
10451 information technology manager european operation
10452 finance manager technology it
10453 desktop engineering manager
10454 global it consumption leader
10455 technology leader
10456 group manager ecu systems test
10457 digital transformation manager
10458 manager application support
10459 slamdunk project manager
10460 sap global projects manager
10461 software engineer tech team lead
10462 project manager pecking order project
10463 signal processing group manager
10464 regional practise manager
10465 global consulting manager
10466 technology project lead
10467 onsite program manager
10468 lead data scientist innovative strategies transformation
10469 technical product manager mm
10470 data science practice lead global
10471 general manager hk macau south china
10472 general manager european sales marketing division
10473 marketing analytics business lead group analytics
10474 regional marketing manager panemeia
10475 technical service

11450 crm erp team leader
11451 independent consultant program leadmanagement consultant
11452 technical lead contract
11453 bandleader
11454 regulatory affairs manager
11455 sales manager uk
11456 software development manager – corporate information systems
11457 global business leader custom contract reagents
11458 finance manager hanergy
11459 internal it manager
11460 digital data manager
11461 area manager south europe
11462 project leader systems engineer
11463 data scientist team lead
11464 manager consumer products localization
11465 e business applications manager
11466 country manager belux signify
11467 global customer experience manager
11468 service planning assurance manager
11469 global manager software
11470 lead business consultant smart sustainable cities
11471 business intelligence architect product manager consultant
11472 manager data management geomatics americas
11473 manager it operations project management
11474 project group manager
11475 programm manager ehea

12449 data analytics manager
12450 technology career manager
12451 national practice manager bi analytics
12452 portfolio manager contract
12453 global it strategy innovation leader
12454 it infrastructure service manager
12455 general manager dba
12456 strategy consulting manager
12457 project manager business transformation it
12458 big data solutions architect tech leader
12459 lead risk analyst analytics infrastructure
12460 projectmanager strategy innovation
12461 product tech lead
12462 deputy general manager
12463 national retail sales manager
12464 associate business performance manager
12465 engineering it project management team – manager
12466 manager strategy
12467 architect team leader lead programmer software developer
12468 innovation project lead
12469 plant general manager completion production solutions
12470 enterprise data management lead europe
12471 manufacturing manager highend vehicles
12472 high voltage projects group manager
12473 division manager information 

13449 regional lead websphere pervasive computing
13450 teknikchef technical manager
13451 lead iphone apple tv developer atp tennis tv
13452 group wide leadership development manager
13453 shader developershading supervisor
13454 programmamanager
13455 database manager sql server technical lead
13456 manager architecture strategy framework systems
13457 product manager it
13458 bu manager emea oilfieldlubricant
13459 manager sales planning analysis
13460 project leader global strategy business development
13461 business process analysis manager
13462 general manager corporate strategy
13463 manager amazon marketplace
13464 solutions practice manager
13465 projektleder project manager
13466 emea gotomarket lead application solutions
13467 section manager cea enterprise architecture global it
13468 it lead finance artificial intelligenceartificial intelligence automation
13469 business team leader
13470 product manager consulting
13471 leadartificial intelligencedata scientist
13472 glo

14449 responsable projet project manager
14450 developers relation manager
14451 continuous improvement leader
14452 manager health informatics research
14453 manager itfm
14454 regional lead apac
14455 general manager turkey
14456 custom solutions manager
14457 emea industry lead energyutilities manufacturing
14458 supervisor realtime computer systems
14459 group it development manager
14460 lead dw solution architect
14461 deputy project manager canadian surface combatant
14462 program manager smart grid
14463 technology research center manager
14464 manager maintenance software deployment
14465 export sales manager
14466 product management technology consultant service delivery lead
14467 lead software engineer data scientist
14468 marketing manager assistant
14469 worldwide ecommerce manager
14470 engineer manager
14471 strategy architecture manager
14472 manager outpatient rehab – st joseph ’ hospital
14473 manager corporate development planning
14474 research marketing services m

In [227]:
jobs.seniority.value_counts()

                   38966
CXO                19272
manager            17451
director           17163
president          14197
senior             11435
head               10622
vp                  6251
lead                5258
founder             3939
board               3615
project manager     3058
partner             2613
sr                  1837
executive           1622
principal           1564
assistant           1329
intern              1054
owner               1026
associate            927
chairman             654
account manager      636
trainee              459
internship           348
junior               330
supervisor           184
jr                    21
Name: seniority, dtype: int64

In [242]:
jobs.role_clean.nunique()

86063

### Job Seggregation

In [143]:
def get_seggregation(job):
    text = job.lower()
    
    remove_punct = str.maketrans("", "", string.punctuation)
    text = text.translate(remove_punct)
    
    remove_digits = str.maketrans("", "", string.digits)
    text = text.translate(remove_digits)
    
    text = text.replace(' ml ', 'machine learning')
    text = text.replace(' ai ', 'artificial intelligence')
    
    tokens = word_tokenize(text)
    
    stop_words = stopwords.words('english')
    stop_words.remove('it')
    tokens_stop = [y for y in tokens if y not in stop_words]
    text = " ".join(tokens_stop)
    text2, text3 = '', ''
    
    if 'consult' in text or 'advis' in text:
        return 'Consultant'
    
    if 'research'in text:
        return 'Research'
    
    if 'scien'in text:
        return 'Scientist'
    
    if 'engineer'in text:
        return 'Engineering'
    
    if 'develop'in text:
        return 'Development'
    
    if 'architect'in text:
        return 'Architecture'
    
    if 'analy'in text or len(re.findall("business intelligence|\\bbi\\b", text)) > 0 or "insight" in text:
        return 'Analytics'

    return ""

In [150]:
jobs['seg'] = jobs.role.apply(lambda x: get_seggregation1(x))

In [160]:
jobs[jobs.seg == ""].role_clean.value_counts()

chief technology officer                   2243
chief information officer                  1481
cio                                        1283
project manager                            1248
president                                  1201
                                           ... 
ikea group strategy process manager           1
business requirements manager                 1
project manager customer care                 1
head transformation interim                   1
freelance graphic designer print online       1
Name: role_clean, Length: 57775, dtype: int64

In [159]:
jobs.seg.value_counts()

                111159
Consultant       13319
Engineering      10816
Development       9056
Analytics         8532
Architecture      4985
Scientist         4230
Research          3734
Name: seg1, dtype: int64

In [158]:
jobs.to_csv('data/processed/jobs.csv',index=False)